In [ ]:
import pandas as pd
import re

df = pd.read_csv("bot_detection_data.csv")

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+|www\S+", "", text)  # remove URLs
    text = re.sub(r"\s+", " ", text).strip()
    return text

final_df = pd.DataFrame({
    "Text": df["Tweet"].map(clean_text),
    "Bot Label": df["Bot Label"]
})

# remove empty rows
final_df = final_df[final_df["Text"].str.len() > 0]

# remove duplicates
final_df = final_df.drop_duplicates(subset=["Text"]).reset_index(drop=True)

print(final_df.shape)
final_df.head(10000)